### projection model => project points per player  
- [ ]    Basic Stats  
- [ ]    Usage  
- [ ]    Percentage of team scoring  
- [ ]    Injuries  
- [ ]    Bye week
- [ ]    Injuries to Key Teammates
- [ ]    Individual Matchups
- [ ]    Opposing Team (record, defense, injuries, etc.)
- [ ]    Home/Away
- [ ]    Weather
- [ ]    Expert Picks
- [x]    Other Fantasy Projections

### selection mechanism => choose optimal team within parameters
- [ ]    maximize value (pts/salary)
- [x]    Linear Optimization
    - Inspiration: https://github.com/breeko/Fantasy_LP/blob/master/fantasy_lp_final.ipynb

In [1]:
import pandas as pd
# from matplotlib import pyplot as plt
import numpy as np

# Get Data

In [2]:
year = 2024
week = 10

In [3]:
primary_dir = f"/Users/adamschiller/OneDrive - BOOZ ALLEN HAMILTON/"
sal_dir = f"{primary_dir}/Weekly Salary Data"

data_dir = "2024_fantasy_data"

## BettingPros - fantasy prop bets

In [198]:
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from io import StringIO

In [199]:
def last_page(driver):
    page_info = driver.find_element(By.CLASS_NAME, "typography.pbcs-page-pagination__text")
    cur_page, pages = re.findall(r'\d+', page_info.text)
    print("Current Page: ", cur_page)
    return int(cur_page) == int(pages)

def get_page_data(driver):
    "return current page player data as list"
    res = []
    lines = driver.find_elements(By.CLASS_NAME, "grouped-items-with-sticky-footer__content")
    
    for line in lines:
        r = line.text.split('\n')
        res.append({'name': r[1], 'prop': r[3], 'projected': r[5].split(' ')[-1]})
    return res

def next_page(driver):
    # next page
    child = driver.find_element(By.CLASS_NAME, "fas.fa-angle-right")
    parent = child.find_element(By.XPATH, "./../..")
    
    # clicking didn't work (overlapping element) but executing script to click does...
    # parent.click()
    driver.execute_script("arguments[0].click();", parent)

def get_bettingpros_data():
    # current week
    # url = "https://www.bettingpros.com/nfl/picks/prop-bets/bet/weekly-fantasy-points/"
    url = "https://www.bettingpros.com/nfl/picks/prize-picks/bet/weekly-fantasy-points/"
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    
    data = []
    
    while True:
        data.extend(get_page_data(driver))
        if last_page(driver):
            break
        else:
            next_page(driver)
    driver.close()
    return pd.DataFrame(data)

In [200]:
bpdf = get_bettingpros_data()
bpdf

Current Page:  1
Current Page:  2
Current Page:  3


,name,prop,projected
0,Diontae Johnson,5.5,8.5
1,Bucky Irving,8,10.9
2,Andrei Iosivas,5.5,7.7
3,James Cook,13.5,15.6
4,Jahmyr Gibbs,14,15.9
...,...,...,...
57,Cade Otton,13.5,13.4
58,Jonathan Taylor,17,17.1
59,Khalil Shakir,12.5,12.4
60,Tua Tagovailoa,16.5,16.6


## Fantasy Sharks

In [19]:
segment = 818 + week
url = f"https://www.fantasysharks.com/apps/bert/forecasts/projections.php?League=-1&Position=99&scoring=16&Segment={segment}&uid=4"
url

'https://www.fantasysharks.com/apps/bert/forecasts/projections.php?League=-1&Position=99&scoring=16&Segment=828&uid=4'

In [20]:
def split_name(x):
    res = x.split(', ')
    return res[1] + " " + res[0]

In [160]:
def fantasysharks_csv(filepath):
    df = pd.read_csv(filepath)
    df.rename(columns={'Player Name': 'name', 'Position': 'pos', 'Pts': 'prediction'}, inplace=True)
    df = df[df['pos'].isin(["QB", "WR", "RB", "TE", "D"])]
    df['name'] = df['name'].apply(split_name)
    df.loc[df['pos']=='D', 'pos'] = 'DST'
    df = df[['name','pos','prediction']]
    return df

shark_df = fantasysharks_csv(f"{data_dir}/week{week}_projections.csv")

In [161]:
shark_df

,name,pos,prediction
0,Justin Jefferson,WR,23.8
1,Lamar Jackson,QB,23.3
2,Jalen Hurts,QB,22.9
3,Josh Allen,QB,22.1
4,Joe Burrow,QB,21.1
...,...,...,...
1008,Brandon Allen,QB,0.0
1009,Carson Wentz,QB,0.0
1010,Trevor Siemian,QB,0.0
1011,Kyle Trask,QB,-0.1


In [162]:
shark_df['pos'].value_counts()

pos
WR     149
RB     103
TE      85
QB      67
DST     28
Name: count, dtype: int64

## NFL

In [ ]:
# nfl_pos_idx = c("QB" = 1, "RB" = 2, "WR" = 3, "TE" = 4, "K" = 7, "DST" = 8)
# pos_scrape = nfl_pos_idx[pos]

# url = "https://fantasy.nfl.com/research/projections"

## ESPN

In [ ]:
# https://fantasy.espn.com/football/players/projections

## CBS

In [141]:
def fix_dumb_team_names(x):
    if 'N.Y.' in x:
        return x.replace('N.Y.', 'New York')
    elif 'L.A.' in x:
        return x.replace('L.A.', 'Los Angeles')
    else:
        return x

In [142]:
def defense_mapping(df):
    return {' '.join(n.split(' ')[:-1]): n for n in df[df['pos']=='DST']['name']}

opp_defense_map = defense_mapping(adf)

def apply_opp_defense_map(x):
    x = fix_dumb_team_names(x)
    try:
        return opp_defense_map[x]
    except KeyError:
        return x

In [143]:
def pos_df(pos):
    df = pd.read_html(f"https://www.cbssports.com/fantasy/football/stats/{pos}/{year}/{week}/projections/nonppr/")[0]
    df.columns = df.columns.droplevel(0)

    # convert to .5 PPR
    rec = df['rec  Receptions']/2 if 'rec  Receptions' in df else 0
    df['prediction'] = df['fppg  Fantasy Points Per Game'] + rec

    # fix names
    if 'Player' in df:
        df['name'] =  df['Player'].apply(lambda x: x.split(pos)[1].strip().split("  ")[-1] if pos in x else np.nan)
        df.dropna(subset=['name'], inplace=True)
    if 'Team' in df:
        df['name'] = df['Team'].apply(lambda x: apply_opp_defense_map(x))

    df['pos'] = pos
    df = df[['name','pos','prediction']]
    return df

In [163]:
positions = ["QB", "WR", "RB", "TE", "DST"]
final = []
for pos in positions:
    final.append(pos_df(pos))
cbs_df = pd.concat(final)
cbs_df

,name,pos,prediction
0,Lamar Jackson,QB,28.6
1,Jalen Hurts,QB,26.0
2,Josh Allen,QB,25.0
3,Jayden Daniels,QB,24.6
4,Joe Burrow,QB,22.6
...,...,...,...
23,Miami Dolphins,DST,6.9
24,Atlanta Falcons,DST,6.5
25,New Orleans Saints,DST,5.9
26,Cincinnati Bengals,DST,5.5


In [164]:
cbs_df['pos'].value_counts()

pos
WR     100
RB      93
TE      92
QB      56
DST     28
Name: count, dtype: int64

## Rotowire

In [38]:
# https://www.rotowire.com/daily/nfl/dfs-opportunities.php?site=DraftKings  # choose full slate
def rotowire_csv(filepath):
    df = pd.read_csv(filepath)
    df.rename(columns={'PLAYER': 'name', 'POS': 'pos', 'SAL': 'salary', 'FPTS': 'prediction'}, inplace=True)
    df.loc[df['pos']=='D', 'pos'] = 'DST'
    df = df[['name','pos','salary','prediction']]
    return df
    
rdf = rotowire_csv(f"{data_dir}/week{week}_rotowire-NFL-projected-roster-percent.csv")
rdf

,name,pos,salary,prediction
0,Daniel Jones,QB,5300,17.33
1,Justin Herbert,QB,5200,16.32
2,Josh Allen,QB,7700,22.92
3,Aaron Rodgers,QB,5700,16.40
4,Kyler Murray,QB,6300,18.21
...,...,...,...,...
523,Patrick Murtagh,TE,2500,0.00
524,David White,WR,3000,0.00
525,Jordan Travis,QB,4000,0.00
526,Keilan Robinson,RB,4000,0.00


In [40]:
len(rdf[rdf['pos']=='DST'])

28

In [41]:
adf = rdf

## Antonio's Official Data

In [6]:
# df = pd.read_csv(f"{sal_dir}/Week{week}_Salaries_rotowire-NFL-projected-roster-percent.csv")
df = pd.read_csv(f"{sal_dir}/Week{week}_salaries_rotowire.csv")

def format_antonio_salary_data(df):
    df.rename(columns={'PLAYER': 'name', 'SAL': 'salary', 'FPTS': 'prediction', 'POS': 'pos'}, inplace=True)
    # df.loc[df['pos']=='DST', 'pos'] = 'Def'
    df.loc[df['pos']=='D', 'pos'] = 'DST'
    df = df[['name','pos','salary','prediction']]
    return df

adf = format_antonio_salary_data(df)
adf

KeyError: 'pos'

In [ ]:
len(adf[adf['pos']=='DST'])

## Daily Fantasy Fuel

In [47]:
# map defense names for dff based on rotowire names
def defense_mapping(df):
    return {n.split(' ')[-1]: n for n in df[df['pos']=='DST']['name']}

defense_map = defense_mapping(adf)

def apply_defense_map(x):
    try:
        return defense_map[x]
    except KeyError:
        return x

In [48]:
# https://www.dailyfantasyfuel.com/nfl/projections/
def dff_csv(filepath):
    df = pd.read_csv(filepath)
    df['name'] = df['first_name']+' '+df['last_name']
    df.rename(columns={'position': 'pos', 'ppg_projection': 'prediction'}, inplace=True)
    # fix names for defenses based on mapping created by rotowire data above
    df.loc[df['pos']=='DST', 'name'] = df['first_name'].apply(lambda x: apply_defense_map(x))
    # df.loc[df['pos']=='DST', 'pos'] = 'Def'
    df = df[['name','pos','injury_status','salary','prediction']]
    return df

dff = dff_csv(f"{data_dir}/week{week}_DFF_NFL_cheatsheet.csv")
dff

,name,pos,injury_status,salary,prediction
0,Lamar Jackson,QB,NaN,8000,24.5
1,Josh Allen,QB,NaN,7700,22.8
2,Jalen Hurts,QB,NaN,7800,21.5
3,Ja'Marr Chase,WR,NaN,8400,21.2
4,Jayden Daniels,QB,NaN,7500,21.0
...,...,...,...,...,...
408,Dak Prescott,QB,O,6100,0.0
409,Trevor Lawrence,QB,O,5400,0.0
410,Colton Dowell,WR,O,3000,0.0
411,Robert Burns,RB,O,4000,0.0


In [58]:
dff['pos'].value_counts()

pos
WR     147
RB     124
TE      81
QB      33
DST     28
Name: count, dtype: int64

# Merge sources

In [165]:
def remove_name_suffixes(df):
    df['name'] = df['name'].str.replace(r'\s?(Jr|Sr|III).?', '', regex=True)
    return df

In [166]:
name_map = {
    'Chig Okonkwo': 'Chigoziem Okonkwo',
    'D.J. Moore': 'DJ Moore',
    'Gabriel Davis': 'Gabe Davis',
    'Josh Palmer': 'Joshua Palmer',
}

def apply_name_map(x):
    if x in name_map:   
        print("fixed", x)
        return name_map[x]
    else:
        return x

def standardize_names(df):
    # remove suffixes
    df['name'] = df['name'].str.replace(r'\s?(Jr|Sr|III).?', '', regex=True)
    # apply map
    df['name'] = df['name'].apply(apply_name_map)
    return df

In [168]:
shark_df = standardize_names(shark_df)
cbs_df = standardize_names(cbs_df)
adf = standardize_names(adf)
dff = standardize_names(dff)

fixed Chig Okonkwo


In [208]:
bpdf = standardize_names(bpdf)
bpdf.rename(columns={'prop': 'prediction_prop', 'projected': 'prediction'}, inplace=True)
bpdf['prediction_prop'] = bpdf['prediction_prop'].astype(float)
bpdf['prediction'] = bpdf['prediction'].astype(float)
bpdf

,name,prediction_prop,prediction
0,Diontae Johnson,5.5,8.5
1,Bucky Irving,8.0,10.9
2,Andrei Iosivas,5.5,7.7
3,James Cook,13.5,15.6
4,Jahmyr Gibbs,14.0,15.9
...,...,...,...
57,Cade Otton,13.5,13.4
58,Jonathan Taylor,17.0,17.1
59,Khalil Shakir,12.5,12.4
60,Tua Tagovailoa,16.5,16.6


In [ ]:
# TODO: determine which names don't match -> outer merge

def find_unmatched_names(df1, df2, thresh_col='prediction', thresh=5):
    "find unmatched names based on some column threshold value"
    merged_df = pd.merge(df1, df2, on='name', how='outer', indicator=True)
    
    # Filter for rows that are only present in one DataFrame
    mismatches = merged_df[merged_df['_merge'] != 'both']
    return mismatches[(mismatches[thresh_col+'_x']>5) | (mismatches[thresh_col+'_y']>5)]

In [169]:
find_unmatched_names(adf, dff)

,name,pos_x,salary_x,prediction_x,pos_y,injury_status,salary_y,prediction_y,_merge


In [184]:
from typing import List, Union
import pandas as pd
from functools import reduce

def merge_dataframes(*dfs: pd.DataFrame, 
                    on: Union[str, List[str]], 
                    how: str = 'outer',
                    rename_duplicates: bool = True,
                    suffix_pattern: str = '_{i}') -> pd.DataFrame:
    """
    Merge multiple pandas DataFrames on specified column(s) while handling duplicate columns.
    
    Args:
        *dfs: Variable number of pandas DataFrames to merge
        on: Column name(s) to merge on. Can be a string or list of strings
        how: Type of merge to perform ('outer', 'inner', 'left', 'right'). Defaults to 'outer'
        rename_duplicates: If True, renames overlapping columns with suffixes before merging.
                         If False, uses pandas default behavior of _x, _y suffixes
        suffix_pattern: Pattern for suffix when rename_duplicates=True. 
                       Use {i} as placeholder for index
    
    Returns:
        pd.DataFrame: Merged DataFrame
        
    Raises:
        ValueError: If less than 2 DataFrames are provided or if merge column is missing
        TypeError: If inputs are not pandas DataFrames
    """
    # Input validation
    if len(dfs) < 2:
        raise ValueError("At least 2 DataFrames are required for merging")
    
    if not all(isinstance(df, pd.DataFrame) for df in dfs):
        raise TypeError("All arguments must be pandas DataFrames")
    
    # Convert single string to list for consistent handling
    merge_cols = [on] if isinstance(on, str) else on
    
    # Verify merge columns exist in all DataFrames
    for i, df in enumerate(dfs):
        missing_cols = set(merge_cols) - set(df.columns)
        if missing_cols:
            raise ValueError(f"DataFrame at index {i} is missing merge columns: {missing_cols}")
    
    if rename_duplicates:
        renamed_dfs = []
        # Keep track of all column names we've seen (except merge columns)
        all_cols = set()
        
        for i, df in enumerate(dfs):
            df = df.copy()
            # Get columns that aren't merge columns
            non_merge_cols = [col for col in df.columns if col not in merge_cols]
            
            # Find overlapping columns
            overlapping = set(non_merge_cols) & all_cols
            
            # Rename overlapping columns
            if overlapping:
                rename_dict = {col: f"{col}{suffix_pattern.format(i=i)}" 
                             for col in overlapping}
                df.rename(columns=rename_dict, inplace=True)
            
            # Add these columns to our tracking set
            all_cols.update(non_merge_cols)
            renamed_dfs.append(df)
        
        # Use renamed DataFrames for merging
        dfs_to_merge = renamed_dfs
    else:
        dfs_to_merge = dfs
    
    # Perform the merge using reduce
    try:
        merged = reduce(lambda left, right: pd.merge(left, right, 
                                                   on=merge_cols, 
                                                   how=how), 
                       dfs_to_merge)
        return merged
    
    except Exception as e:
        raise Exception(f"Error during merge operation: {str(e)}")

In [210]:
merged = merge_dataframes(
    adf, 
    dff[['name','injury_status','prediction']], 
    shark_df[['name','prediction']], 
    cbs_df[['name','prediction']],
    bpdf,
    on='name', how='left'
)
# remove injured
merged = merged[merged['injury_status'].isna()]
merged = merged.drop(columns=['injury_status'])
# merged = merged.fillna(0.0)
# get mean prediction (disregard when value isn't present)
merged['mean_pred'] = merged.filter(like='prediction').replace(0, np.nan).mean(axis=1)
# filter out preds w/ fewer than 3 values
merged['pred_count'] = merged.filter(like='prediction').gt(0).sum(axis=1)
merged = merged[merged['pred_count']>=3]
merged

,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count
0,Daniel Jones,QB,5300,17.33,17.3,15.2,19.80,16.0,17.1,17.121667,6
1,Justin Herbert,QB,5200,16.32,16.4,16.1,18.80,NaN,NaN,16.905000,4
2,Josh Allen,QB,7700,22.92,22.8,22.1,25.00,20.5,21.7,22.503333,6
3,Aaron Rodgers,QB,5700,16.40,16.9,13.8,18.60,16.0,16.5,16.366667,6
4,Kyler Murray,QB,6300,18.21,18.6,16.0,19.40,16.5,17.8,17.751667,6
...,...,...,...,...,...,...,...,...,...,...,...
431,Jonathan Ward,RB,4000,0.00,0.1,0.2,0.05,NaN,NaN,0.116667,3
471,Adam Prentice,RB,4000,0.00,0.3,0.2,0.05,NaN,NaN,0.183333,3
488,Mason Rudolph,QB,4900,0.00,10.8,10.3,3.40,NaN,NaN,8.166667,3
523,Israel Abanikanda,RB,4000,0.16,0.2,0.9,NaN,NaN,NaN,0.420000,3


## Original

In [120]:
def merge_sources(df1, df2):
    df = df1.merge(df2[['name','injury_status','prediction']], on='name', how='inner', suffixes=('_df1', '_df2'))
    df = df[df['injury_status'].isna()]
    df = df[(df['prediction_df1']>0)&(df['prediction_df2']>0)]
    df['prediction'] = df[['prediction_df1', 'prediction_df2']].mean(axis=1)
    df['pred_diff'] = abs(df['prediction_df1']-df['prediction_df2'])
    df.drop(columns=['injury_status'], inplace=True)  #,'prediction_df1','prediction_df2'
    return df

In [121]:
df = merge_sources(adf, dff)
df

,name,pos,salary,prediction_df1,prediction_df2,prediction,pred_diff
0,Daniel Jones,QB,5300,17.33,17.3,17.315,0.03
1,Justin Herbert,QB,5200,16.32,16.4,16.360,0.08
2,Josh Allen,QB,7700,22.92,22.8,22.860,0.12
3,Aaron Rodgers,QB,5700,16.40,16.9,16.650,0.50
4,Kyler Murray,QB,6300,18.21,18.6,18.405,0.39
...,...,...,...,...,...,...,...
361,Hassan Haskins,RB,4000,0.46,0.6,0.530,0.14
362,Malik Washington,WR,3000,0.33,1.1,0.715,0.77
363,Tip Reiman,TE,2500,0.33,0.3,0.315,0.03
370,Cody Schrader,RB,4000,0.17,0.2,0.185,0.03


In [122]:
df['pos'].value_counts()

pos
WR     112
RB      82
TE      72
DST     28
QB      26
Name: count, dtype: int64

## XDF -- Additional Data Sources

In [170]:
find_unmatched_names(cbs_df, shark_df)

,name,pos_x,prediction_x,pos_y,prediction_y,_merge
38,Brandin Cooks,NaN,NaN,WR,6.1,right_only
75,Chigoziem Okonkwo,NaN,NaN,TE,6.0,right_only
78,Chris Olave,NaN,NaN,WR,9.4,right_only
208,Jameson Williams,WR,7.6,NaN,NaN,left_only
401,Tee Higgins,NaN,NaN,WR,10.6,right_only
417,Trenton Irwin,NaN,NaN,WR,5.5,right_only
418,Trevor Lawrence,NaN,NaN,QB,15.3,right_only


In [125]:
cbs_df['prediction_df3'] = cbs_df['prediction']
shark_df['prediction_df4'] = shark_df['prediction']

xdf = df.merge(cbs_df[['name', 'prediction_df3']], on='name', how='inner')
xdf = xdf.merge(shark_df[['name', 'prediction_df4']], on='name', how='inner')

In [55]:
xdf['prediction'] = xdf[['prediction_df1', 'prediction_df2', 'prediction_df3', 'prediction_df2']].mean(axis=1)

In [78]:
# cbs_df[cbs_df['name'].str.contains('Trevor')]
# shark_df[shark_df['name'].str.contains('Tyrone')]

,name,pos,prediction


In [81]:
find_unmatched_names(cbs_df, shark_df)

,name,pos_x,prediction_x,pos_y,prediction_y,_merge
38,Brandin Cooks,NaN,NaN,WR,6.1,right_only
46,Brian Robinson,NaN,NaN,RB,12.4,right_only
47,Brian Robinson Jr.,RB,8.70,NaN,NaN,left_only
77,Chigoziem Okonkwo,NaN,NaN,TE,6.0,right_only
80,Chris Olave,NaN,NaN,WR,9.4,right_only
102,D.J. Moore,NaN,NaN,WR,13.5,right_only
104,DJ Moore,WR,10.80,NaN,NaN,left_only
160,Gabe Davis,WR,5.15,NaN,NaN,left_only
161,Gabriel Davis,NaN,NaN,WR,8.2,right_only
212,Jameson Williams,WR,7.60,NaN,NaN,left_only


In [56]:
xdf['pos'].value_counts()

pos
TE     69
RB     67
WR     66
QB     26
DST     4
Name: count, dtype: int64

# Top Projected per Pos

In [215]:
def avg_per_dollar(df, col):
    df['Avg Value'] = df[col] / (df['salary'] / 1000)
    return df.sort_values('Avg Value', ascending=False)

pdf = avg_per_dollar(merged, 'mean_pred')

In [216]:
top_QBs = pdf[pdf.pos=='QB']
top_QBs.head(5)

,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count,Avg Value
1,Justin Herbert,QB,5200,16.32,16.4,16.1,18.8,NaN,NaN,16.905000,4,3.250962
0,Daniel Jones,QB,5300,17.33,17.3,15.2,19.8,16.0,17.1,17.121667,6,3.230503
7,Brock Purdy,QB,6500,19.05,19.6,21.0,20.2,18.5,19.7,19.675000,6,3.026923
10,Lamar Jackson,QB,8000,22.63,24.5,23.3,28.6,23.0,23.2,24.205000,6,3.025625
5,Sam Darnold,QB,6200,17.80,18.6,17.6,19.0,NaN,NaN,18.250000,4,2.943548


In [217]:
top_RBs = pdf[pdf.pos=='RB']
top_RBs.head(5)

,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count,Avg Value
24,Chase Brown,RB,6000,15.32,18.4,14.0,15.25,15.5,16.0,15.745000,6,2.624167
39,De'Von Achane,RB,7100,17.09,17.2,17.3,17.05,19.0,18.5,17.690000,6,2.491549
49,Alvin Kamara,RB,8100,18.75,20.4,18.0,18.05,19.5,20.1,19.133333,6,2.362140
48,Bijan Robinson,RB,7700,17.33,18.5,19.0,17.65,NaN,NaN,18.120000,4,2.353247
22,Derrick Henry,RB,8200,21.30,19.8,19.4,17.65,16.5,18.1,18.791667,6,2.291667


In [218]:
top_WRs = pdf[pdf.pos=='WR']
top_WRs.head(5)

,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count,Avg Value
35,DeAndre Hopkins,WR,5300,12.69,12.6,13.8,9.90,NaN,NaN,12.247500,4,2.310849
54,Ja'Marr Chase,WR,8400,18.57,21.2,19.9,16.85,18.5,20.2,19.203333,6,2.286111
112,Justin Jefferson,WR,8800,17.65,19.2,23.8,16.25,NaN,NaN,19.225000,4,2.184659
46,Tyreek Hill,WR,7500,16.84,17.4,16.8,14.30,15.5,16.9,16.290000,6,2.172000
60,Garrett Wilson,WR,7200,15.86,16.0,15.3,15.05,14.5,16.3,15.501667,6,2.153009


In [219]:
top_TEs = pdf[pdf.pos=='TE']
top_TEs.head(5)

,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count,Avg Value
9,Mike Gesicki,TE,3600,10.03,14.5,9.9,9.15,11.5,11.3,11.063333,6,3.073148
37,Mark Andrews,TE,4100,9.79,11.6,12.1,9.35,10.5,9.3,10.440000,6,2.546341
26,George Kittle,TE,5800,15.20,15.4,16.4,13.25,13.0,14.7,14.658333,6,2.527299
28,Travis Kelce,TE,6000,14.85,15.6,13.4,11.65,NaN,NaN,13.875000,4,2.312500
42,Hunter Henry,TE,3900,8.95,9.1,9.6,7.20,NaN,NaN,8.712500,4,2.233974


In [27]:
# players[players.Pos == 'Def'].head(3)
top_Defs = pdf[pdf.pos=='Def']
top_Defs.head(5)

,name,pos,salary,prediction_rdf,prediction_dff,prediction,pred_diff,Avg Value
83,Kansas City Chiefs,Def,3100,8.18,8.7,8.440,0.52,2.722581
115,Miami Dolphins,Def,3500,8.01,9.7,8.855,1.69,2.530000
99,Denver Broncos,Def,3200,7.72,8.4,8.060,0.68,2.518750
95,Minnesota Vikings,Def,3200,7.67,7.8,7.735,0.13,2.417187
92,New England Patriots,Def,3200,7.61,7.7,7.655,0.09,2.392187


# Team Selection

$$\begin{aligned} 
objective &= maximize\ team\ points \\ 
team\ salary &\leq salary\ cap \\
team &= 1\ QB + (2|3)\ RB + (3|4)\ WR + (1|2)\ TE + 1\ DEF \\
team &\leq 9\ total\ players \\
\end{aligned}$$

In [39]:
# %pip install pulp

In [211]:
from pulp import *

class PulpSelection():
    def __init__(self, df, pts_col="prediction", sal_col="salary", name_col="name", salary_cap=50000):
        self.df = df
        self.vars = self.populate_vars(pts_col, sal_col, name_col)
        self.model = self.optimize(salary_cap)
        self.players = self.player_names()
        self.selection = self.selection_df()
        
    def populate_vars(self, pts_col, sal_col, name_col):
        df = self.df
        salaries = {}
        points = {}
        for pos in df.pos.unique():
            available_pos = df[df.pos == pos]
            salary = list(available_pos[[name_col,sal_col]].set_index(name_col).to_dict().values())[0]
            point = list(available_pos[[name_col,pts_col]].set_index(name_col).to_dict().values())[0]
            salaries[pos] = salary
            points[pos] = point
            
        self.salaries = salaries
        self.points = points
        return {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

    def player_names(self):
        players = {}
        for d in self.vars.values():
            for k,v in d.items():
                players[v] = k
        return players
            
    pos_num_available = {
        "QB": 1,
        "RB": 2,
        "WR": 3,
        "TE": 1,
        "DST": 1
    }

    def optimize(self, salary_cap):
        prob = LpProblem("FFModel", LpMaximize)
        rewards = []
        costs = []

        for pos, players in self.vars.items():
            costs += lpSum([self.salaries[pos][i] * self.vars[pos][i] for i in players])
            rewards += lpSum([self.points[pos][i] * self.vars[pos][i] for i in players])
            if pos in ['RB','WR','TE']:
                prob += lpSum([self.vars[pos][i] for i in players]) <= self.pos_num_available[pos]+1
                prob += lpSum([self.vars[pos][i] for i in players]) >= self.pos_num_available[pos]
            else:
                prob += lpSum([self.vars[pos][i] for i in players]) == self.pos_num_available[pos]
        prob += lpSum(prob.variables()) == 9    # flex -> max of 9 total players

        prob += lpSum(rewards)
        prob += lpSum(costs) <= salary_cap
        print(prob.solve())
        return prob
    
    def selection_df(self):
        selections = [self.players[p] for p in self.model.variables() if p.varValue > 0]
        team = self.df[self.df.name.isin(selections)]
        return team

In [212]:
pts_col = 'mean_pred'
model = PulpSelection(merged, pts_col=pts_col)
team = model.selection

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/adamschiller/bah_projects/model_football/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/pl/yvv0vk397x34b_lzgcmsd43w0000gp/T/0f53205181af4e4f96fa53e3b267db03-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/pl/yvv0vk397x34b_lzgcmsd43w0000gp/T/0f53205181af4e4f96fa53e3b267db03-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 2172 RHS
At line 2183 BOUNDS
At line 2500 ENDATA
Problem MODEL has 10 rows, 316 columns and 1208 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 130.355 - 0.00 seconds
Cgl0004I processed model has 7 rows, 298 columns (298 integer (298 of which binary)) and 894 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.117647
Cbc0038I Solution found of -130.107
Cbc0038I Before m

In [213]:
print("\nTotal points: {}     Total salary: {}\n".format(sum(team[pts_col]), sum(team['salary'])))
team


Total points: 130.10666666666665     Total salary: 49900



,name,pos,salary,prediction,prediction_1,prediction_2,prediction_3,prediction_prop,prediction_4,mean_pred,pred_count
9,Mike Gesicki,TE,3600,10.03,14.5,9.9,9.15,11.5,11.3,11.063333,6
10,Lamar Jackson,QB,8000,22.63,24.5,23.3,28.60,23.0,23.2,24.205000,6
20,Chicago Bears,DST,3000,7.77,8.6,10.5,14.30,NaN,NaN,10.292500,4
24,Chase Brown,RB,6000,15.32,18.4,14.0,15.25,15.5,16.0,15.745000,6
35,DeAndre Hopkins,WR,5300,12.69,12.6,13.8,9.90,NaN,NaN,12.247500,4
37,Mark Andrews,TE,4100,9.79,11.6,12.1,9.35,10.5,9.3,10.440000,6
39,De'Von Achane,RB,7100,17.09,17.2,17.3,17.05,19.0,18.5,17.690000,6
54,Ja'Marr Chase,WR,8400,18.57,21.2,19.9,16.85,18.5,20.2,19.203333,6
68,DeMario Douglas,WR,4400,9.68,9.7,NaN,NaN,8.0,9.5,9.220000,4


## Save Picks to CSV

In [214]:
remote_dir = f"{primary_dir}/Weekly Team Picks/Week {week:02}/"
local_dir = "2024_weekly_picks/"
fname = f"Adam_week{week}_picks.csv"

# save version in each location
for d in [local_dir, remote_dir]:
    team[['pos','name','salary']].to_csv(d+fname, index=False)

# Optimal Picks per Week

In [19]:
# salary + scoring
df = df.join(week_df[['name', 'fpts']].set_index('name'), on='name')
df

,name,pos,salary,prediction,fpts
0,Patrick Mahomes,QB,8100,25.420,21.34
1,Joe Burrow,QB,6300,20.025,14.80
2,Trevor Lawrence,QB,6500,20.465,15.74
4,Sam Howell,QB,5500,16.570,18.34
5,Dak Prescott,QB,6200,18.470,NaN
...,...,...,...,...,...
351,Jason Brownlee,WR,3000,0.225,NaN
354,Brandon Bolden,RB,4000,0.365,NaN
355,Davis Allen,TE,2500,0.195,NaN
356,Mike Boone,RB,4000,0.580,1.10


In [20]:
def show_optimal_picks(df, pts_col='fpts', sal_col='salary'):
    week = df.dropna()
    model = PulpSelection(week, pts_col=pts_col)
    team = model.selection
    print("\nTotal points: {}     Total salary: {}\n".format(sum(team[pts_col]), sum(team[sal_col])))
    return team

In [21]:
show_optimal_picks(df)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/Caskroom/miniconda/base/envs/default/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/cv/pyjmd76x58dbyxrhzycj6c880000gp/T/277c43891ee94cf3b082240ccce60552-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/cv/pyjmd76x58dbyxrhzycj6c880000gp/T/277c43891ee94cf3b082240ccce60552-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 1690 RHS
At line 1701 BOUNDS
At line 1952 ENDATA
Problem MODEL has 10 rows, 250 columns and 947 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 242.62 - 0.00 seconds
Cgl0004I processed model has 7 rows, 230 columns (230 integer (226 of which binary)) and 690 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -242.62
Cbc0038I Cleaned solution of -242.62
C

,name,pos,salary,prediction,fpts
11,Drake London,WR,4800,12.345,24.50
28,Jared Goff,QB,6400,16.830,25.42
63,Raheem Mostert,RB,6400,15.740,37.20
67,Cooper Kupp,WR,9000,20.780,30.80
69,Kyren Williams,RB,6500,15.255,24.80
88,Tyreek Hill,WR,9300,22.715,31.30
90,Adam Thielen,WR,5900,13.610,31.50
92,Minnesota Vikings,Def,3000,6.075,21.00
97,Dalton Schultz,TE,3600,8.195,16.10


# Scoring submission

In [42]:
def get_scores():
    res = []
    for pos in ['QB','RB','WR','TE','dst']:
        url = (f"https://fantasydata.com/nfl/fantasy-football-leaders?scope=game"
               f"&sp={year}_REG&week_from={week}&week_to={week}&position={pos}"
                "&scoring=fpts_draftkings&order_by=fpts_draftkings&sort_dir=desc")
        df = pd.read_html(url)[0]
        if pos == 'dst':
            df = df.rename(columns={'TEAM': 'name'})
            df['pos'] = 'DST'
        else:
            df.columns = df.columns.droplevel(0)
        df = df.rename(columns=str.lower)
        res.append(df[['name','pos','fpts']])
    return pd.concat(res)

In [ ]:
df = get_scores()
df

In [45]:
local_dir = "2024_weekly_picks/"
fname = f"Adam_week{week}_picks.csv"
team = pd.read_csv(local_dir+fname)
team

,pos,name,salary
0,TE,Mike Gesicki,3600
1,QB,Lamar Jackson,8000
2,DST,Chicago Bears,3000
3,RB,Chase Brown,6000
4,WR,DeAndre Hopkins,5300
5,TE,Mark Andrews,4100
6,RB,De'Von Achane,7100
7,WR,Ja'Marr Chase,8400
8,WR,DeMario Douglas,4400


In [47]:
def score_submission(df, score_df):
    df = df.join(score_df[['name', 'fpts']].set_index('name'), on='name')  # join in scores dataframe
    df = df.fillna(0)  # fill empties with 0
    print(f"Total Points in Week {week}:", sum(df['fpts']))
    return df

In [48]:
team = score_submission(team, df)

Total Points in Week 10: 163.1


In [49]:
team

,pos,name,salary,fpts
0,TE,Mike Gesicki,3600,7.0
1,QB,Lamar Jackson,8000,32.9
2,DST,Chicago Bears,3000,4.0
3,RB,Chase Brown,6000,23.4
4,WR,DeAndre Hopkins,5300,9.6
5,TE,Mark Andrews,4100,18.8
6,RB,De'Von Achane,7100,0.0
7,WR,Ja'Marr Chase,8400,58.4
8,WR,DeMario Douglas,4400,9.0
